# Tutorial 1
In this tutorial, we are going to build an SNN capable of classifying MNIST by copying the weights obtained by training the following simple ANN using TensorFlow:

Using GeNN for spike-based machine learning.svg

Clearly, this is far from a state of the art architecture, but it still achieves 97.6% accuracy on MNIST.
## Install PyGeNN wheel from Google Drive
Download wheel file

In [1]:
!gdown 1LMVTqYWWQdidyKKX-bKT-0EFTnzdFnm5

Downloading...
From: https://drive.google.com/uc?id=1LMVTqYWWQdidyKKX-bKT-0EFTnzdFnm5
To: /content/pygenn-4.9.0-cp310-cp310-linux_x86_64.whl
100% 20.6M/20.6M [00:00<00:00, 145MB/s]


and then install PyGeNN from wheel file

In [2]:
!pip install pygenn-4.9.0-cp310-cp310-linux_x86_64.whl

Processing ./pygenn-4.9.0-cp310-cp310-linux_x86_64.whl


and checkout latest mlGeNN release from git and install

In [3]:
!rm -rf /content/ml_genn
!git clone https://github.com/genn-team/ml_genn.git --branch ml_genn_2_1 -c advice.detachedHead=false
!pip install ./ml_genn/ml_genn

Cloning into 'ml_genn'...
remote: Enumerating objects: 7154, done.
remote: Counting objects: 100% (2971/2971), done.
remote: Compressing objects: 100% (1134/1134), done.
remote: Total 7154 (delta 1875), reused 2829 (delta 1810), pack-reused 4183
Receiving objects: 100% (7154/7154), 37.20 MiB | 20.93 MiB/s, done.
Resolving deltas: 100% (4720/4720), done.
Processing ./ml_genn/ml_genn
  Preparing metadata (setup.py) ... done
  Created wheel for ml-genn: filename=ml_genn-2.0.0-py3-none-any.whl size=102336 sha256=a795b658b4685499fb5fc0ebcf05abd49be42e14bf675f9f1922fc2242ce7bd2
  Stored in directory: /tmp/pip-ephem-wheel-cache-gtyk18bu/wheels/3f/cf/27/0e9dec4bb1be2afac4b38c2dfb4ce0bc164ce1ecb32b6f91b8
Successfully built ml-genn


Set environment variable to allow GeNN to find CUDA

In [4]:
%env CUDA_PATH=/usr/local/cuda

env: CUDA_PATH=/usr/local/cuda


## Download pre-trained weights

In [5]:
!gdown 1cmNL8W0QZZtn3dPHiOQnVjGAYTk6Rhpc
!gdown 131lCXLEH6aTXnBZ9Nh4eJLSy5DQ6LKSF

Downloading...
From: https://drive.google.com/uc?id=1cmNL8W0QZZtn3dPHiOQnVjGAYTk6Rhpc
To: /content/weights_0_1.npy
100% 402k/402k [00:00<00:00, 141MB/s]
Downloading...
From: https://drive.google.com/uc?id=131lCXLEH6aTXnBZ9Nh4eJLSy5DQ6LKSF
To: /content/weights_1_2.npy
100% 5.25k/5.25k [00:00<00:00, 25.3MB/s]


## Install MNIST package

In [6]:
!pip install mnist

## Build model
Import standard modules and required mlGeNN classes

In [7]:
import mnist
import numpy as np

from ml_genn import InputLayer, Layer, SequentialNetwork
from ml_genn.compilers import InferenceCompiler
from ml_genn.neurons import IntegrateFire, IntegrateFireInput
from ml_genn.connectivity import Dense
from ml_genn.callbacks import SpikeRecorder

Because our network is entirely feedforward, we can define it as a ``SequentialNetwork`` where each layer is automatically connected to the previous layer. We are going to convert MNIST digits to spikes by simply treating the intensity of each pixel (multiplied by a scaling factor) as the input to an integrate-and-fire neuron. For our hidden and output layers we are going use very simple Integrate-and-Fire neurons which best match the transfer function of the ReLU neurons our ANN was trained with. Finally, we are going to read classifications from our output layer by simply counting spikes.



In [8]:
# Create sequential model
network = SequentialNetwork()
with network:
    input = InputLayer(IntegrateFireInput(v_thresh=5.0), 784)
    Layer(Dense(weight=np.load("weights_0_1.npy")),
          IntegrateFire(v_thresh=5.0))
    output = Layer(Dense(weight=np.load("weights_1_2.npy")),
                   IntegrateFire(v_thresh=5.0, readout="spike_count"))

In mlGeNN, in order to turn an abstract network description into something that can actually be used for training or inference you use a *compiler* class. Here, as we are performing inference, we use the InferenceCompiler and specify batch size and how many timesteps to evaluate each example for.

In [9]:
compiler = InferenceCompiler(dt=1.0, batch_size=128,
                             evaluate_timesteps=100)
compiled_net = compiler.compile(network)

We can then load the MNIST testing data and evaluate the compiled pre-trained network on it. The normal grayscale values of the MNIST dataset are passed into the input layer, scaled by a factor (here ```0.01```) which is translated into spikes by the ```IntegrateFireInput``` layer. The testing labels are passed as a simple array of of ```0``` to ```9```, corresponding to the default ```SparseCategorical``` loss type.

In [10]:
# Load testing data
testing_images = np.reshape(mnist.test_images(), (-1, 784))
testing_labels = mnist.test_labels()

with compiled_net:
    # Evaluate model on numpy dataset
    metrics, _ = compiled_net.evaluate({input: testing_images * 0.01},
                                       {output: testing_labels})


  0%|          | 0/79 [00:00<?, ?it/s]